## Dataframe Basics

In [1]:
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [47.2 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,284 kB]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,521 kB]
Hit:13 https://ppa.launchpadcontent

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrame Basics").getOrCreate()


In [3]:
# Create DataFrame manually
dataframe = spark.createDataFrame([
    (0, "Here is our DataFrame"),
    (1, "We are making one from scratch"),
    (2, "This will look very similar to a Pandas DataFrame")
], ["id", "words"])

dataframe.show()


+---+--------------------+
| id|               words|
+---+--------------------+
|  0|Here is our DataF...|
|  1|We are making one...|
|  2|This will look ve...|
+---+--------------------+



In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/1/food.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("food.csv"), sep=",", header=True)

# Show DataFrame
df.show()


+-------+-----+
|   food|price|
+-------+-----+
|  pizza|    0|
|  sushi|   12|
|chinese|   10|
+-------+-----+



In [5]:
# Print our schema


root
 |-- food: string (nullable = true)
 |-- price: string (nullable = true)



In [6]:
# Show the columns


['food', 'price']

In [7]:
# Describe our data


+-------+-------+-----------------+
|summary|   food|            price|
+-------+-------+-----------------+
|  count|      3|                3|
|   mean|   NULL|7.333333333333333|
| stddev|   NULL|6.429100507328637|
|    min|chinese|                0|
|    max|  sushi|               12|
+-------+-------+-----------------+



In [8]:
# Import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, StructType


In [9]:
# Next we need to create the list of struct fields
schema = [StructField("food", StringType(), True), StructField("price", IntegerType(), True),]


[StructField('food', StringType(), True),
 StructField('price', IntegerType(), True)]

In [10]:
# Pass in our fields


StructType([StructField('food', StringType(), True), StructField('price', IntegerType(), True)])

In [11]:
# Read our data with our new schema


+-------+-----+
|   food|price|
+-------+-----+
|  pizza|    0|
|  sushi|   12|
|chinese|   10|
+-------+-----+



In [12]:
# Print it out


root
 |-- food: string (nullable = true)
 |-- price: integer (nullable = true)



### Accessing data

Column<'price'>

pyspark.sql.column.Column

DataFrame[price: int]

pyspark.sql.dataframe.DataFrame

+-----+
|price|
+-----+
|    0|
|   12|
|   10|
+-----+



### Manipulating Columns

In [18]:
# Add new column


+-------+-----+--------+
|   food|price|newprice|
+-------+-----+--------+
|  pizza|    0|       0|
|  sushi|   12|      12|
|chinese|   10|      10|
+-------+-----+--------+



In [19]:
# Update column name


+-------+----------+
|   food|newerprice|
+-------+----------+
|  pizza|         0|
|  sushi|        12|
|chinese|        10|
+-------+----------+



In [20]:
# Double the price


+-------+-----+-----------+
|   food|price|doubleprice|
+-------+-----+-----------+
|  pizza|    0|          0|
|  sushi|   12|         24|
|chinese|   10|         20|
+-------+-----+-----------+



In [21]:
# Add a dollar to the price


+-------+-----+--------------+
|   food|price|add_one_dollar|
+-------+-----+--------------+
|  pizza|    0|             1|
|  sushi|   12|            13|
|chinese|   10|            11|
+-------+-----+--------------+



In [22]:
# Half the price


+-------+-----+----------+
|   food|price|half_price|
+-------+-----+----------+
|  pizza|    0|       0.0|
|  sushi|   12|       6.0|
|chinese|   10|       5.0|
+-------+-----+----------+



In [23]:
# Collecting a column as a list


[Row(price=0), Row(price=12), Row(price=10)]

# Converting PySpark DataFrame to Pandas DataFrame

In [24]:
import pandas as pd
pandas_df = dataframe.toPandas()


In [25]:
pandas_df.head()


food  price
0    pizza      0
1    sushi     12
2  chinese     10